Install dependencies

In [2]:
!pip install neo4j
!pip install requests

     |████████████████████████████████| 89 kB 6.8 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.1-py3-none-any.whl size=114783 sha256=78e61276ddf5c22f890581b6524a4f94a60514c715636bb67335c1e1bae66784
  Stored in directory: /root/.cache/pip/wheels/fd/15/02/8379f87426cd1b74fd5891f49df9fb978423feb97650cd639b
Successfully built neo4j


First create the FHIR data types. Need this before anything else.


In [36]:
from neo4j import GraphDatabase
import requests
import json
import urllib.request

driver = GraphDatabase.driver("neo4j://3.83.17.192:7687", auth=("neo4j", "troubleshooters-decreases-helicopters"))
load_file_dir = "https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/"

def clear(tx):
    tx.run("CALL apoc.periodic.iterate('MATCH (n) RETURN n', 'DETACH DELETE n', {batchSize:1000})")

def file_load(load_files):
  with driver.session() as session:
    nodes = []
    relationships = []
    for file_item in load_files:
      filename = "%s%s" % (load_file_dir, file_item["filename"])#
      if "label" in file_item:
        nodes.append("{ fileName: '%s', labels: ['%s'] }" % (filename, file_item["label"]) )
      else:
        relationships.append("{ fileName: '%s', type: '%s' }" % (filename, file_item["type"]) )
    query = """CALL apoc.import.csv( [%s], [%s], {stringIds: false})""" % (", ".join(nodes), ", ".join(relationships))
    print(query)
    result = session.run(query)
    print(result)
    for record in result:
      print(record)
  driver.close()

with driver.session() as session:
    session.write_transaction(clear)
driver.close()

print("Ready ...")


Ready ...


In [37]:
stage_1_files = [ 
    { "label": "FHIR", "filename": "stage_1_fhir_nodes.csv" },
    { "label": "WEB_SOURCE", "filename": "stage_1_web_source_nodes.csv" },
    { "label": "FHIR_DATA_TYPE", "filename": "stage_1_fhir_data_type_nodes.csv" },
    { "label": "FHIR_DATA_TYPE_PROPERTY", "filename": "stage_1_fhir_data_type_property_nodes.csv" },
    { "type": "HAS_DATA_TYPE_PROPERTY", "filename": "stage_1_has_data_type_property_relationships.csv" },
    { "type": "FROM_SOURCE", "filename": "stage_1_from_source_relationships.csv" },
    { "type": "HAS_DATA_TYPE", "filename": "stage_1_has_data_type_relationships.csv" }
  ]

file_load(stage_1_files)

CALL apoc.import.csv( [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_fhir_nodes.csv', labels: ['FHIR'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_web_source_nodes.csv', labels: ['WEB_SOURCE'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_fhir_data_type_nodes.csv', labels: ['FHIR_DATA_TYPE'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_fhir_data_type_property_nodes.csv', labels: ['FHIR_DATA_TYPE_PROPERTY'] }], [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_has_data_type_property_relationships.csv', type: 'HAS_DATA_TYPE_PROPERTY' }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_1_from_source_relationships.csv', type: 'FROM_SOURCE' }, { fileName: 'https://raw.

Now check we have loaded ok. Check the FHIR version, should be 4.0.1

In [38]:
with driver.session() as session:
  query = """MATCH (n:FHIR) Return n.version as version""" 
  result = session.run(query)
  for record in result:
    print("Version: ", record["version"])

driver.close()

Version:  4.0.1


In [39]:
stage_2_files = [ 
    { "label": "CANONICAL_MODEL", "filename": "stage_2_canonical_model_nodes.csv" },
    { "label": "CANONICAL_NODE", "filename": "stage_2_canonical_node_nodes.csv" },
    { "label": "CANONICAL_DATA_TYPE", "filename": "stage_2_canonical_data_type_nodes.csv" },
    { "label": "OTHER_SOURCE", "filename": "stage_2_other_source_nodes.csv" },
    { "type": "CONSISTS_OF", "filename": "stage_2_consists_of_relationships.csv" },
    { "type": "FROM_SOURCE", "filename": "stage_2_from_source_relationships.csv" },
    { "type": "HAS_SUB_MODEL", "filename": "stage_2_has_sub_model_relationships.csv" },
    { "type": "HAS_DATA_TYPE", "filename": "stage_2_has_data_type_relationships.csv" }
]

file_load(stage_2_files)

CALL apoc.import.csv( [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_canonical_model_nodes.csv', labels: ['CANONICAL_MODEL'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_canonical_node_nodes.csv', labels: ['CANONICAL_NODE'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_canonical_data_type_nodes.csv', labels: ['CANONICAL_DATA_TYPE'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_other_source_nodes.csv', labels: ['OTHER_SOURCE'] }], [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_consists_of_relationships.csv', type: 'CONSISTS_OF' }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_2_from_source_relationships.csv', type: 'FROM_SOURCE' }, { fileName: 'https://raw.gith

Check a few relationships exist in the canonical model. Should get three results.

* THERAPEUTIC INTERVENTION
* OBSERVATION
* ADVERSE EVENT



In [40]:
with driver.session() as session:
  query = """MATCH (r)-[]->(t)-[]->(n:CANONICAL_NODE) WHERE n.name="LOCATION" RETURN r.name as root_name, t.name as name""" 
  result = session.run(query)
  for record in result:
    print("%s -> %s -> Location: " % (record["root_name"], record["name"]))

driver.close()

CANONICAL MODEL -> ADVERSE EVENT -> Location: 
CANONICAL MODEL -> THERAPEUTIC INTERVENTION -> Location: 
CANONICAL MODEL -> OBSERVATION -> Location: 


Now link the canonical nodes and the data types using the name as the key.

In [41]:
with driver.session() as session:
  query = """MATCH (n:CANONICAL_NODE)-[]->(m:CANONICAL_DATA_TYPE) RETURN m.uri as uri""" 
  result = session.run(query)
  for record in result:
    print(record["uri"])
    query = """MATCH (n:CANONICAL_DATA_TYPE{uri:'%s'})
      WITH n
      MATCH (dt:FHIR_DATA_TYPE{name: n.data_type})
      CREATE (n)-[:IS_A]->(dt)
      RETURN n.data_type as cn_name, dt.name as dt_name;
    """ % (record["uri"])
    inner_result = session.run(query)
    for inner_record in inner_result:
      print("%s, %s linked" % (inner_record["cn_name"], inner_record["dt_name"]))

driver.close()

http://id.d4k.dk/dataset/canonical/n8/coding
coding, coding linked
http://id.d4k.dk/dataset/canonical/n9/coding
coding, coding linked
http://id.d4k.dk/dataset/canonical/n18/coding
coding, coding linked


Now create the data type nodes for the canonical mode. For each canonical node that references a data type copy the properties for that data type to the canonical node. Give each canonical leaf a unique id. These nodes should also have a C code reference providig a definition for the data item.


In [42]:
uri_data = []
with driver.session() as session:
  query = """MATCH (n:CANONICAL_DATA_TYPE)-[:IS_A]->(dt:FHIR_DATA_TYPE) RETURN n.uri as canonical, dt.uri as data_type""" 
  result = session.run(query)
  for record in result:
    query = """MATCH (n:CANONICAL_DATA_TYPE{uri:'%s'}), (dt:FHIR_DATA_TYPE{uri:'%s'})
      CALL apoc.path.subgraphAll(dt, {relationshipFilter:'HAS_DATA_TYPE_PROPERTY>'})
      YIELD nodes, relationships
      CALL apoc.refactor.cloneSubgraph(
        nodes,
        [rel in relationships WHERE type(rel) = 'HAS_DATA_TYPE_PROPERTY'],
        { standinNodes:[[dt, n]] })
      YIELD input, output, error
      RETURN output;
    """ % (record["canonical"], record["data_type"])
    inner_result = session.run(query)
    for inner_record in inner_result:
      node = inner_record["output"]
      uri_data.append({"id": node.id, "uri": "%s#%s" % (record["canonical"], node["name"]) })
    print ("Duplicated for: ", record["canonical"])

driver.close()

with driver.session() as session:
  query = """UNWIND $uri_data AS d
    MATCH (n) WHERE ID(n)=d.id
    SET n.uri = d.uri;""" 
  result = session.run(query, uri_data=uri_data)
driver.close()
print ("URIs set.")



Duplicated for:  http://id.d4k.dk/dataset/canonical/n18/coding
Duplicated for:  http://id.d4k.dk/dataset/canonical/n9/coding
Duplicated for:  http://id.d4k.dk/dataset/canonical/n8/coding
URIs set.


In [43]:
stage_3_files = [ 
    { "label": "API_SOURCE", "filename": "stage_3_api_source_nodes.csv" },
    { "label": "SKOS_CONCEPT", "filename": "stage_3_skos_concept_nodes.csv" },
    { "label": "SKOS_CONCEPT_SCHEME", "filename": "stage_3_skos_concept_scheme_nodes.csv" },
    { "type": "HAS_SOURCE", "filename": "stage_3_from_source_relationships.csv" },
    { "type": "SKOS_HAS_TOP_CONCEPT", "filename": "stage_3_skos_has_top_concept_relationships.csv" },
    { "type": "SKOS_NARROWER", "filename": "stage_3_skos_narrower_relationships.csv" }
  ]

file_load(stage_3_files)


CALL apoc.import.csv( [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_api_source_nodes.csv', labels: ['API_SOURCE'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_skos_concept_nodes.csv', labels: ['SKOS_CONCEPT'] }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_skos_concept_scheme_nodes.csv', labels: ['SKOS_CONCEPT_SCHEME'] }], [{ fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_from_source_relationships.csv', type: 'HAS_SOURCE' }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_skos_has_top_concept_relationships.csv', type: 'SKOS_HAS_TOP_CONCEPT' }, { fileName: 'https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/stage_3_skos_narrower_relationships.csv', type: 'SKOS_NARROWER' }], {stringIds: false})

Do a quick test on the CT. Check AGEU and its links

In [44]:
with driver.session() as session:
  query = """MATCH (cs)-[]->(c1:SKOS_CONCEPT)-[]->(c2) WHERE c1.notation = 'AGEU' RETURN DISTINCT cs.version as version, c1.identifier as cl_identifier, c2.identifier as cli_identifier, c2.notation as cli_notation""" 
  result = session.run(query, uri_data=uri_data)
  for record in result:
    print ("%s: %s, %s, %s" % (record["version"], record["cl_identifier"], record["cli_identifier"], record["cli_notation"]))
driver.close()


2021-12-17: C66781, C29846, MONTHS
2021-12-17: C66781, C29848, YEARS
2021-12-17: C66781, C29844, WEEKS
2021-12-17: C66781, C25301, DAYS
2021-12-17: C66781, C25529, HOURS


Stage 4 to 9 are the CT files for the other areas, ADaM, Protocol, CDASH etc. Not loaded at the moment. Needed to split due to size and limited RAM on the Neo4j server.